---
**Introduction to Numerical Shooting**

Week 15 of EMAT30008

https://engmaths.github.io/emat30008/odes/numericalshooting/ 

(See week15funcs.py for associated functions)

Author: Ben Winstanley, yy23737@bristol.ac.uk

---

In [ ]:
import numpy as np
import scipy as sp 
import matplotlib.pyplot as plt
import time